# SCRAP COURSERA

In [218]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import traceback

In [219]:
#subjects
#subjects = ['Business', 'Computer Science', 'Data Science','Health', 'Social Sciences','Physical Science and Engineering','Information Technology', 'Arts and Humanities', 'Language Learning', 'Personal Development', 'Math and Logic']
subjects = ['Business' ,'Physical Science and Engineering']

#course tab on edx.org
base_website = 'https://www.coursera.org/search'

#query string as per subject
# eg for Business, https://www.coursera.org/search?index=prod_all_launched_products_term_optimization&entityTypeDescription=Courses&topic=Business
website = 'https://www.coursera.org/search?index=prod_all_launched_products_term_optimization&entityTypeDescription=Courses&topic={courseName}'

# maximum number of courses to parse per subject
MAX_COURSES_PER_SUBJECT = 12

In [220]:
# return url for the passed on subject
def subjectUrl(subName):
    subName = subName.replace(" ","%20").replace("&","%26")
    return website.format(courseName=subName)

In [221]:
## NOTE: Configure WebDriver and append webdriver's location into PATH system variable

# driver = webdriver.Chrome()
driver = webdriver.Edge()

### Data 
Description of the data frame fields

| field   | Description                           |
| ------- | ------------------------------------- |
| subject | Subject of course                     |
| title   | Course Title                          |
| desc    | Course Description                    |
| outcome | What you'll learn                     |
| skills  | Skills you'll gain                    |
| hours   | Average hours per week for completion |
| instit  | Hosting Institute                     |
| level   | Difficulty level of course            |
| lang    | Language-Subtitle                     |
| url     | URL of course                         |
| imgurl  | Course Image                          |

*Note: For a given course all the above fields might not be present, in that case the values will be populated with "NA"*




In [222]:
lsubject = []
ltitle = []
ldesc = []
loutcome = []
lskills = []
lhours = []
linstit = []
llevel = []
llang = []
lurl = []
limgurl =  []



In [223]:
# return False if element is absent and else text within 
def isPresent(by, xpth, returnList = False):
    lst = driver.find_elements( by ,xpth)
    if ( len(lst) > 0):
        if(returnList):
            return list(map(lambda x: x.get_attribute('textContent'), lst))
        else:
            return lst[0].get_attribute('textContent')
    else:
        return False 

try:
    for subject in subjects:

        #load the subject page
        driver.get(subjectUrl(subject))
        time.sleep(3)

        #Counter cnt
        cnt = 0

        #list course links
        course_list = []

        #list of img urls mapped with above courses
        course_img_list = []

        #do ... while cnt <= MAX_COURSES_PER_SUBJECT and next is enabled
        while(cnt <= MAX_COURSES_PER_SUBJECT):

            iter_list = driver.find_elements(By.XPATH, "/html//main[@id='main']/div[@class='ais-InstantSearch__root']/div/div[@class='rc-SearchTabs']/div[@class='ais-MultiIndex__root']//ul[@class='ais-InfiniteHits-list']//a")

            iter_img_list = driver.find_elements(By.XPATH, "/html//img[@class='product-photo']")

            if ( MAX_COURSES_PER_SUBJECT > cnt + len(iter_list) ):
                #Filter out the urls from elements 
                course_list += list(map(lambda x: str(x.get_attribute('href')), iter_list))
                course_img_list += list(map(lambda x: str(x.get_attribute('src')), iter_img_list))
                cnt += len(iter_list)

            else:
                #Filter out the urls from elements 
                course_list += list(map(lambda x: str(x.get_attribute('href')), iter_list[:(MAX_COURSES_PER_SUBJECT-cnt)]))
                course_img_list += list(map(lambda x: str(x.get_attribute('src')), iter_img_list[:(MAX_COURSES_PER_SUBJECT-cnt)]))
                cnt += MAX_COURSES_PER_SUBJECT-cnt
            
            # NEXT button on course result page
            NEXT_BTN = driver.find_element(By.XPATH,"/html//main[@id='main']/div[@class='ais-InstantSearch__root']//div[@class='rc-SearchTabs']//div[@class='ais-InfiniteHits']/div[1]/div[@role='navigation']/button[7]")
            
            # if cnt<MAX_COURSES_PER_SUBJECT and NEXT_BTN is enabled
            if(cnt<MAX_COURSES_PER_SUBJECT and NEXT_BTN.is_enabled()):
                NEXT_BTN.click()
                time.sleep(2)
            else:
                break

        # Subject and count
        print("\nSubject: ", subject)
        print("Count: ",cnt)
        #print(course_list)
        #print(course_img_list)

        
        for course_count in range(cnt):

            driver.get(course_list[course_count])

            #tit = driver.find_element_by_xpath("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[@class='header']//h1").text
            tit = isPresent(By.XPATH, "/html//main[@id='main']/div//div[@id='main']/div[2]/h1[@class='banner-title banner-title-without--subtitle m-b-0']")
            
            des = isPresent(By.XPATH, "/html//div[@class='m-t-1 description']")

            out = isPresent(By.XPATH, "/html//main[@id='main']/div/div[2]//div[@class='m-y-2']/div[@class='CmlLearningObjectives border-a p-x-2 p-t-1']/ul/*", True)

            ski = isPresent(By.XPATH, "/html//main[@id='main']/div/div[2]//div[@class='Skills m-y-2 p-x-2 p-t-1 p-b-2 border-a css-1rj0z6b']/ul[@role='list']/*", True)

            tim = isPresent(By.XPATH, "/html//main[@id='main']/div/div[2]/div[@class='_1d3lkver p-b-2']//div[@class='ProductGlance']/div[last()-1]//span")
            
            ins = isPresent(By.XPATH, "/html//main[@id='main']/div/div[4]/div[@class='_1d3lkver']//div[@class='_wtdnuob']/a/h3")

            lvl = isPresent(By.XPATH, "/html//main[@id='main']/div/div[2]/div[@class='_1d3lkver p-b-2']//div[@class='ProductGlance']/div[last()-2]/div[2]")

            lan = isPresent(By.XPATH, "/html//main[@id='main']/div/div[2]/div[@class='_1d3lkver p-b-2']//div[@class='ProductGlance']/div[last()]/div[2]/*", True)

            img = course_img_list[course_count]

            #out, ski, lan are list
            #print(tit, des, out, ski, tim, ins, lvl, lan, img, sep='\n')
            
            
            # clean and append or omit if absent 
            tim = tim.replace('Approx. ','').replace(' hours to complete','')
            lvl = lvl.replace(' level', '')

            #separate medium language and subtitles by "-"
            lan = lan[0] + lan[1].replace('Subtitles: ', '-')

            if( out ):
                out = ". ".join(out)
            else:
                out = "NA"

            if( ski ):
                ski = ". ".join(ski)
            else:
                ski = "NA"
            
            #print(tit, des, out, ski, tim, ins, lvl, lan, img, sep='\n')
            # append values into list
            lsubject.append(subject)
            ltitle.append(tit)
            ldesc.append(des)
            loutcome.append(out)
            lskills.append(ski)
            lhours.append(tim)
            linstit.append(ins)
            llevel.append(lvl)
            llang.append(lan)
            lurl.append(course_list[course_count])
            limgurl.append(img)
            
except Exception:
    print('ERROR: Process interupted due to an error at url', driver.current_url)
    print(traceback.format_exc())
    


Subject:  Business
Count:  12

Subject:  Physical Science and Engineering
Count:  12


In [224]:
#closing the browser
driver.close()

In [225]:
df = pd.DataFrame({'subject': lsubject, 'title': ltitle, 'description': ldesc, 'outcomes': loutcome, 'skills': lskills, 'hours': lhours, 'institute': linstit, 'level':llevel, 'Languages': llang, 'url': lurl, 'img_url': limgurl})
df.to_csv('cousera.csv')
df

,subject,title,description,outcomes,skills,hours,institute,level,Languages,url,img_url
0,Business,Foundations of Project Management,This course is the first in a series of six to...,"Describe project management skills, roles, and...",Organizational Culture. Career Development. St...,15,Google,Beginner Level,English-English,https://www.coursera.org/learn/project-managem...,https://d3njjcbhbojbot.cloudfront.net/api/util...
1,Business,Financial Markets,"An overview of the ideas, methods, and institu...",NA,Behavioral Finance. Financial Markets. Finance...,33,Yale University,Beginner Level,"English-Arabic, French, Portuguese (European),...",https://www.coursera.org/learn/financial-marke...,https://d3njjcbhbojbot.cloudfront.net/api/util...
2,Business,AI For Everyone,AI is not only for engineers. If you want your...,NA,Workflow of Machine Learning projects. AI term...,12,DeepLearning.AI,Beginner Level,"English-Chinese (Traditional), Arabic, French,...",https://www.coursera.org/learn/ai-for-everyone,https://d3njjcbhbojbot.cloudfront.net/api/util...
3,Business,Project Initiation: Starting a Successful Project,This is the second course in the Google Projec...,Understand the significance of the initiation ...,Stakeholder Management. Strategic Thinking. Bu...,19,Google,Beginner Level,English-English,https://www.coursera.org/learn/project-initiat...,https://d3njjcbhbojbot.cloudfront.net/api/util...
4,Business,Fundamentals of Project Planning and Management,Projects are all around us. Virtually every or...,"How to initiate, define and organize a project...",Risk Analysis. Project Planning. Planning. Man...,8,University of Virginia,Beginner LevelNo prior experience needed to pa...,"English-Arabic, French, Portuguese (European),...",https://www.coursera.org/learn/uva-darden-proj...,https://d3njjcbhbojbot.cloudfront.net/api/util...
5,Business,Successful Negotiation: Essential Strategies a...,We all negotiate on a daily basis. On a person...,NA,Strategic Negotiations. Communication. Negotia...,17,University of Michigan,100% onlineStart instantly and learn at your o...,"English-Arabic, French, Ukrainian, Portuguese ...",https://www.coursera.org/learn/negotiation-skills,https://d3njjcbhbojbot.cloudfront.net/api/util...
6,Business,"Brand Management: Aligning Business, Brand and...",Professor Nader Tavassoli of London Business S...,NA,Corporate Branding. Brand Marketing. Brand Man...,17,University of London,Beginner Level,"English-Arabic, French, Portuguese (European),...",https://www.coursera.org/learn/brand,https://d3njjcbhbojbot.cloudfront.net/api/util...
7,Business,Impact Measurement & Management for the SDGs,How can businesses and investors help fill the...,NA,NA,9,Duke University,100% onlineStart instantly and learn at your o...,English-English,https://www.coursera.org/learn/impact-for-sdgs,https://d3njjcbhbojbot.cloudfront.net/api/util...
8,Business,Private Equity and Venture Capital,The course deals with the analysis of the priv...,NA,Valuation (Finance). Venture Capital. Venture ...,10,Università Bocconi,100% onlineStart instantly and learn at your o...,"English-Arabic, French, Portuguese (European),...",https://www.coursera.org/learn/private-equity,https://d3njjcbhbojbot.cloudfront.net/api/util...
9,Business,Project Planning: Putting It All Together,This is the third course in the Google Project...,Describe the components of the project plannin...,Project Planning. Strategic Thinking. Risk Man...,25,Google,Beginner Level,English-English,https://www.coursera.org/learn/project-plannin...,https://d3njjcbhbojbot.cloudfront.net/api/util...
